In [9]:
import numpy as np
import pandas as pd
import twint
import time
from datetime import datetime

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import nest_asyncio
nest_asyncio.apply()

# PATHS

In [4]:
SAVE_PATH_ROOT = 'PLACEHOLDER'    #folder where non-CT tweets from date ranges will be saved

# Functions

In [13]:
def clean_tweets(df):
    local_df = df
    
    #drop redundant cols
    local_df.drop(['created_at', 'timezone', 'cashtags', 'user_id_str', 'urls', 'photos', 'video', 'thumbnail', 'translate', 'trans_src', 'trans_dest', 'name', 'search'], axis=1, inplace=True)
    
    #convert date to datetime
    local_df['date'] = pd.to_datetime(local_df['date'])
    
    return local_df

In [1]:
def find_tweets_in_date_range(search_terms, start_date, end_date, LOCAL_SAVE_PATH_ROOT):
    '''    
    search_terms = list of strings (or singleton) of search terms that will be in the tweet
    
    returns a pandas dataframe of tweets
    '''
    
    search_string = " OR ".join(search_terms)
    
    c = twint.Config()
    
#     date range of search
    c.Since = start_date
    c.Until = end_date
    
    c.Search = search_string
    
    # only collect a certain number of tweets
#     c.Limit = num_tweets

    # Don't print output
    c.Hide_output = True
    
    # find shadow-banned accounts too - this apparently slows things down considerably
    c.Profile_full = True

#     c.Output = f"{num_tweets} tweets - {start_date} to {end_date}.csv"
    c.Pandas = True

    twint.run.Search(c)

#     tweets_df = twint.storage.panda.Tweets_df
    
#     return clean_tweets(tweets_df)

    clean_tweets(twint.storage.panda.Tweets_df).to_csv(fr"{LOCAL_SAVE_PATH_ROOT}{start_date} to {end_date}.csv")

## Create Date Range

In [ ]:
date_ranges = []

today_str = date.today().strftime('%d-%m-%Y')

for date in pd.date_range(start='1-1-2020', end=today_str, freq='1W'):
    date_tuple = ((date.strftime('%Y-%m-%d'), (date+sixdays).strftime('%Y-%m-%d') ))
    
    print(date_tuple, '\n')
    
    date_ranges.append(date_tuple)

# Find associated hashtags

## LIST OF NON-CT KEYWORDS/HASHTAGS

In [2]:
non_CT_words_and_tags = [
    'essentialworkers',
    'pfizerproud',
    'vaccineswork',
    'sciencewillwin'
]

non_CT_words_and_tags = non_CT_words_and_tags + [f'#{x}' for x in non_CT_words_and_tags]

# Apply Searching Function

In [ ]:
for date_tup in date_ranges:
    find_tweets_in_date_range(search_terms=non_CT_words_and_tags, start_date=date_tup[0], end_date=date_tup[1], LOCAL_SAVE_PATH_ROOT=SAVE_PATH_ROOT)